In [45]:
from itertools import combinations, repeat
from collections import Counter
from typing import List, Tuple, NamedTuple, Optional
from __future__ import annotations
import re


import black
import jupyter_black

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [121]:
# Day 1: Inverse Captcha
def match(digits, offset=1):
    for i, a in enumerate(digits):
        if a == digits[(i + offset) % len(digits)]:
            yield a


digits = open("2017/1.txt").read().strip()
digits = [int(x) for x in digits]

print("Part 1:", sum(match(digits)))  # 1203
print("Part 2:", sum(match(digits, offset=len(digits) // 2)))  # 1146

Part 1: 1203
Part 1: 1146


In [149]:
# Day 2: Corruption Checksum
def evenly_divides(numbers: List[int]) -> int:
    for a, b in combinations(sorted(numbers), 2):
        if b / a == b // a:
            return b // a


lines = open("2017/2.txt").read().strip().splitlines()
spreadsheet = [[int(x) for x in line.split()] for line in lines]
print("Part 1:", sum(max(row) - min(row) for row in spreadsheet))  # 32121
print("Part 2:", sum(evenly_divides(row) for row in spreadsheet))  # 197

Part 1: 32121
Part 2: 197


In [41]:
# Day 3: Spiral Memory
def spiral_move():
    #                         58  57
    # 37  36  35  34  33  32  31  56
    # 38  17  16  15  14  13  30  55
    # 39  18   5   4   3  12  29  54
    # 40  19   6   1   2  11  28  53
    # 41  20   7   8   9  10  27  52
    # 42  21  22  23  24  25  26  51
    # 43  44  45  46  47  48  49  50

    # How to generate:
    # r u l l d d r r r (10) u u u l l l l (17) d d d d (21) r r r r r (26) u u u u u (31)
    # l l l l l l (37) d d d d d d (43) r r r r r r r (50) u u u u u u u (57)

    # 1R, 1U, 2L, 2D, 3R, 3U, 4L, 4D, 5R, 5U, 6L, 6D, 7R, 7U,...

    def move(dirs):
        nonlocal x, y, steps
        for dir in dirs:
            for dy, dx in repeat(dir, steps):
                x += dx
                y += dy
                yield (x, y)
        steps += 1

    right_up = ((1, 0), (0, -1))
    left_down = ((-1, 0), (0, 1))
    steps = 1
    x = y = 0
    yield (x, y)
    while True:
        yield from move(right_up)
        yield from move(left_down)


def distance(N=277678):
    for number, node in enumerate(spiral_move(), 1):
        if number == N:
            return sum(abs(coord) for coord in node)


def value_exceeds(N=277678):
    # fmt:off
    def neighbors8(point): 
        "The eight neighboring squares."
        x, y = point 
        return ((x-1, y-1), (x, y-1), (x+1, y-1),
                (x-1, y),             (x+1, y),
                (x-1, y+1), (x, y+1), (x+1, y+1))
    # fmt:on

    memory = {}
    for node in spiral_move():
        number = sum(memory[n] for n in neighbors8(node) if n in memory) or 1
        memory[node] = number
        if number > N:
            return number


print("Part 1:", distance())  # 475
print("Part 2:", value_exceeds())  # 279138

Part 1: 475
Part 2: 279138


In [19]:
# Day 4: High-Entropy Passphrases
def valid1(line):
    return len(line.split()) == len(set(line.split()))


def valid2(line):
    words = [''.join(sorted(x)) for x in line.split()]
    return len(words) == len(set(words))


lines = open("2017/4.txt").read().strip().splitlines()
print("Part 1:", sum(valid1(line) for line in lines)) # 386
print("Part 2:", sum(valid2(line) for line in lines)) # 208

Part 1: 386
Part 2: 208


In [47]:
# Day 5: A Maze of Twisty Trampolines, All Alike
def run(instructions, part2=False):
    pc = steps = 0
    while pc < len(instructions):
        offset = instructions[pc]
        if part2 and offset >= 3:
            instructions[pc] -= 1
        else:
            instructions[pc] += 1
        steps += 1
        pc += offset
    return steps

instructions = open('2017/5.txt').read().strip().splitlines()
instructions = [int(x) for x in instructions]

print('Part 1:', run(instructions[:])) # 342669
print('Part 2:', run(instructions, part2=True)) # 25136209


Part 1: 342669
Part 2: 25136209


In [76]:
# Day 6: Memory Reallocation
def redistribute(banks: Tuple[int, ...]) -> Tuple[int, ...]:
    banks = list(banks)
    blocks = max(banks)
    index = banks.index(blocks)
    banks[index] = 0
    while blocks:
        index = (index + 1) % len(banks)
        banks[index] += 1
        blocks -= 1
    return tuple(banks)


def cycle_unique(banks: Tuple[int, ...]) -> Tuple[int, int]:
    seen = {}
    cycles = 0
    while banks not in seen:
        seen[banks] = cycles
        banks = redistribute(banks)
        cycles += 1
    return cycles, seen[banks]


banks = tuple(int(x) for x in open("2017/6.txt").read().strip().split())

cycles, first_seen = cycle_unique(banks)
print("Part 1:", cycles)  # 4074
print("Part 2:", cycles - first_seen)  # 2793

Part 1: 4074
Part 2: 2793


In [83]:
# Day 7: Recursive Circus
def parse_lines(lines):
    programs = {}
    children = {}
    parents = {}
    # Parse
    for line in lines:
        items = line.split(" -> ")
        name, weight = items[0].split()
        weight = int(re.search("\d+", weight)[0])
        programs[name] = weight
        if len(items) > 1:
            children[name] = set(items[1].split(", "))
            for child in children[name]:
                parents[child] = name
    return programs, children, parents


def root(programs, parents):
    for program in programs:
        if program not in parents:
            return program


lines = open("2017/7.txt").read().strip().splitlines()
# lines = open("2017/example-7.txt").read().strip().splitlines()
programs, children, parents = parse_lines(lines)
print("Part 1:", root(programs, parents))

Part 1: hlhomy


In [87]:
def branch_weight(node):
    if node not in children:
        return programs[node]
    return sum(branch_weight(child) for child in children[node]) + programs[node]


def unbalanced_weight(node):
    "Returns weight of unbalanced branch, 0 if all branches are balanced."
    branch_weights = Counter(branch_weight(child) for child in children[node])
    if len(branch_weights) == 1:
        return 0
    return branch_weights.most_common()[-1][0]


def node_with_weight(weight):
    "Returns node where branch weight equals `weight`"
    for node in programs:
        if branch_weight(node) == weight:
            return node


def unbalanced_node(node):
    while unbalanced_weight(node):
        node = node_with_weight(unbalanced_weight(node))
    return node


node = root(programs, parents)
node = unbalanced_node(node)

In [91]:
[branch_weight(child) for child in children[parents[node]]]

[1571, 1571, 1571, 1579]

In [94]:
programs[node] - 8

1505